In [ ]:
import os
import SimpleITK as sitk
import nibabel as nib
import shutil
import numpy as np
import matplotlib.pyplot as plt

### Load data to view shape

In [ ]:
id = 384
path = f"..\data\ImageCAS\images\image_{id:04d}.nii.gz"
image_raw = sitk.ReadImage(path)
image = sitk.GetArrayFromImage(image_raw)
print(image.shape)

label_path = f"..\data\ImageCAS\labels\label_{id:04d}.nii.gz"
label_raw = sitk.ReadImage(label_path)
label = sitk.GetArrayFromImage(label_raw)
print(label.shape)

In [ ]:
'''
275  206
206  206
206  257
257  206
206  275
275  275
275  206
'''

In [ ]:
print(np.unique(label))

### Check shape of images and mask

In [ ]:
for index in range(89, 382, 1):
    path = f"../data/ImageCAS/images/image_{index:04d}.nii.gz"
    image_raw = sitk.ReadImage(path)
    image = sitk.GetArrayFromImage(image_raw)
    
    label_path = f"../data/ImageCAS/labels/label_{index:04d}.nii.gz"
    label_raw = sitk.ReadImage(label_path)
    label = sitk.GetArrayFromImage(label_raw)
    
    print(image.shape, label.shape)
    if (image.shape != label.shape): break
    else: print(f"Successful: {index}")

In [ ]:
path = f"../data/imageTBAD/images/image_0098.nii.gz"
image_raw = sitk.ReadImage(path)
image = sitk.GetArrayFromImage(image_raw)
image.shape

### Rename function

In [ ]:
directory = "../data/imageTBAD/labels/"
start_index = 2
end_index = 192

for index in range(start_index, end_index+1, 1):
    filename = f"{index}_label.nii.gz"
    if (os.path.exists(os.path.join(directory, filename))):
        new_filename = f"label_{index:04d}.nii.gz"
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
        print(f"Renamed {filename} to {new_filename}")

In [ ]:
# Increment
directory = "../data/imageTBAD/images/"
start_index = 100
end_index = 393

index = end_index
while index != start_index:
    filename = f"ct_{index:04d}_label.nii.gz"
    if os.path.exists(os.path.join(directory, filename)):
        new_index = index + 1
        new_filename = f"ct_{new_index:04d}_label.nii.gz"
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
        print(f"Renamed {filename} to {new_filename}")
    index -= 1

In [ ]:
# Decrement
directory = "../data/ImageCAS/images/"
start_index = 91
end_index = 384

for index in range(start_index, end_index + 1):
    filename = f"image_{index:04d}.nii.gz"
    if os.path.exists(os.path.join(directory, filename)):
        new_index = index - 1
        new_filename = f"image_{new_index:04d}.nii.gz"
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
        print(f"Renamed {filename} to {new_filename}")

### Mapping names

In [ ]:
directory = '../data/ImageCAS/'
image_filenames = os.listdir(directory + 'images')
label_filenames = os.listdir(directory + 'labels')

len(image_filenames), len(label_filenames)

In [ ]:
for index in range(len(label_filenames)):
    label_filename = label_filenames[index]
    directory_label_filename = directory + f'labels/{label_filename}'
    
    image_filename = image_filenames[index]
    new_filename = f'label_{image_filename[5:]}'
    new_directory_filename = directory + f'labels/{new_filename}'
    
    os.rename(directory_label_filename, new_directory_filename)
    print(f"{index} - Rename: {directory_label_filename} to {new_directory_filename}")

### Seperate labels

In [ ]:
for index in range(1, 107, 1):
    label_path = f"..\data\ImageCHD\mask\ct_{index:04d}_label.nii.gz"
    label_raw = sitk.ReadImage(label_path)
    label = sitk.GetArrayFromImage(label_raw)
    
    unique_vals = np.unique(label)
    if len(unique_vals) != 8: 
        print(f"{index} - error label: {unique_vals}") 
    else: print(f"Successful: {index}")

In [ ]:
def save_func(out_path, volume):
    array = np.array(volume, dtype=np.float32)
    array = np.transpose(array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(array, affine)
    nib.save(nifti_file, out_path)

def save_sep_label(id, mask):
    for val in unique_vals:
        label_array = np.copy(mask)
        label_array[np.where(label_array != val)] = 0 
        save_func(f"../data/ImageCHD/splitted_mask/ct_00{id}_label/ct_00{id}_label_{val+1}.nii.gz", label_array)
        
save_sep_label(
    id = '02',
    mask=label
)

### Visualize data

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(10,10))

for index, ax in enumerate(axes.flat):
    ax.imshow(image[index], cmap='gray')
    ax.axis('off')
    ax.set_title(f'Slice {index+1}')

plt.suptitle("Display data")
plt.tight_layout()
plt.show()

### Processing Data

In [ ]:
# processing tbad
directory = '../data/imageTBAD/'
image_dir = directory + 'images/'
label_dir = directory + 'labels/'

os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

for filename in os.listdir(directory):
    if filename.endswith('_image.nii.gz'): 
        shutil.copy(
            os.path.join(directory, filename),
            os.path.join(image_dir, filename)
        )
    elif filename.endswith('_label.nii.gz'):
        shutil.copy(
            os.path.join(directory, filename),
            os.path.join(label_dir, filename)
        )